# RDD basics

This notebook will introduce three basic but essential Spark operations. Two of them are the *transformations* `map` and `filter`. The other is the *action* `collect`. At the same time we will introduce the concept of *persistence* in Spark.    

## Getting the data and creating the RDD

As we did in our first notebook, we will use the dataset provided for the KDD Cup 2010, containing nearly eight hundred thousand records of algebra-2005-2006. The file is provided as a zip file that we will download locally.

Now we can use this file to create our RDD.

In [ ]:
import findspark
findspark.init()

In [ ]:
import pyspark
sc = pyspark.SparkContext(appName="KDDCup10")

data_file = "algebra_2005_2006/algebra_2005_2006_train.txt"
raw_data = sc.textFile(data_file)

## The `filter` transformation

This transformation can be applied to RDDs in order to keep just elements that satisfy a certain condition. More concretely, a function is evaluated on every element in the original RDD. The new resulting RDD will contain just those elements that make the function return `True`.

For example, imagine we want to count how many `EG4-FIXED` problem Name Records we have in our dataset. We can filter our `raw_data` RDD as follows.  

In [ ]:
eg4_raw_data = raw_data.filter(lambda x: 'EG4-FIXED' in x)

Now we can count how many elements we have in the new RDD.

In [ ]:
from time import time
t0 = time()
eg4_count = eg4_raw_data.count()
tt = time() - t0
print("There are {} 'EG4-FIXED' Problem Name records".format(eg4_count))
print("Count completed in {} seconds".format(round(tt,3)))

Remember from notebook 1 that we have a total of 809695 records in algebra_2005_2006_train dataset. Here we can see that 518 records has the Problem Name equals `EG4-FIXED` tag word.  

Notice that we have measured the elapsed time for counting the elements in the RDD. We have done this because we wanted to point out that actual (distributed) computations in Spark take place when we execute *actions* and not *transformations*. In this case `count` is the action we execute on the RDD. We can apply as many transformations as we want on a our RDD and no computation will take place until we call the first action that, in this case takes a few seconds to complete.

## The `map` transformation

By using the `map` transformation in Spark, we can apply a function to every element in our RDD. Python's lambdas are specially expressive for this particular.

In this case we want to read our data file as a TSV formatted one. We can do this by applying a lambda function to each element in the RDD as follows.

In [ ]:
from pprint import pprint
tsv_data = raw_data.map(lambda x: x.split("\t"))
t0 = time()
head_rows = tsv_data.take(5)
tt = time() - t0
print("Parse completed in {} seconds".format(round(tt,3)))
pprint(head_rows[0])

Again, all action happens once we call the first Spark *action* (i.e. *take* in this case). What if we take a lot of elements instead of just the first few?  

In [ ]:
print(head_rows[4][14])

In [ ]:
t0 = time()
head_rows = tsv_data.take(100000)
tt = time() - t0
print("Parse completed in {} seconds".format(round(tt,3)))

We can see that it takes longer. The `map` function is applied now in a  distributed way to a lot of elements on the RDD, hence the longer execution time.

### Using `map` and predefined functions

Of course we can use predefined functions with `map`. Imagine we want to have each element in the RDD as a key-value pair where the key is the tag (e.g. *EG4-FIXED*) and the value is the whole list of elements that represents the row in the TSV formatted file. We could proceed as follows.    

In [ ]:
def parse_record(line):
    elems = line.split("\t")
    tag = elems[3]
    return (tag, elems)

key_tsv_data = raw_data.map(parse_record)
head_rows = key_tsv_data.take(5)
pprint(head_rows[1])

That was easy, wasn't it?

In our notebook about working with key-value pairs we will use this type of RDDs to do data aggregations (e.g. count by key).

## The `collect` action

So far we have used the actions `count` and `take`. Another basic action we need to learn is `collect`. Basically it will get all the elements in the RDD into memory for us to work with them. For this reason it has to be used with care, specially when working with large RDDs.  

An example using our raw data.    

In [ ]:
t0 = time()
all_eg4_data = eg4_raw_data.collect()
tt = time() - t0
print("Data collected in {} seconds".format(round(tt,3)))

That took longer as any other action we used before, of course. Every Spark worker node that has a fragment of the RDD has to be coordinated in order to retrieve its part, and then *reduce* everything together.    

As a last example combining all the previous, we want to collect all the `EG4-FIXED` records as key-value pairs.   

In [ ]:
# get data from file
data_file = "algebra_2005_2006/algebra_2005_2006_train.txt"
raw_data = sc.textFile(data_file)

# parse into key-value pairs
key_tsv_data = raw_data.map(parse_record)

# filter EG4_FIXED Problem Name key records
eg4_records = key_tsv_data.filter(lambda x: 'EG4-FIXED' in x)

# collect all
t0 = time()
all_eg4 = eg4_records.collect()
tt = time() - t0
eg4_count = len(all_eg4)
print("Data collected in {} seconds".format(round(tt,3)))
print("There are {} 'EG4-FIXED' Problem Names".format(eg4_count))

This count matches with the previous count for `EG4-FIXED` records. The new procedure is more time consuming. This is because we retrieve all the data with `collect` and then use Python's `len` on the resulting list. Before we were just counting the total number of elements in the RDD by using `count`.  